In [1]:
import time
import os
import pandas as pd
import numpy as np
import xgboost as xgb

from datetime import datetime

** Read Data **

In [2]:
train = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Train\\Train.csv')
test = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Test_D7W1juQ.csv')
sample_submission = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\sample_submission_cvwryvM.csv')
health_camp_detail = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Train\\Health_Camp_Detail.csv')
patient_profile = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Train\\Patient_Profile.csv')
first_health_camp_attended = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Train\\First_Health_Camp_Attended.csv')
second_health_camp_attended = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Train\\Second_Health_Camp_Attended.csv')
third_health_camp_attended = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Train\\Third_Health_Camp_Attended.csv')
print "train", train.shape
print "test", test.shape
print "health_camp_detail", health_camp_detail.shape
print "patient_profile", patient_profile.shape
print "first_health_camp_attended", first_health_camp_attended.shape
print "second_health_camp_attended", second_health_camp_attended.shape
print "third_health_camp_attended", third_health_camp_attended.shape

train (75278, 8)
test (35249, 8)
health_camp_detail (65, 6)
patient_profile (37633, 11)
first_health_camp_attended (6218, 5)
second_health_camp_attended (7819, 3)
third_health_camp_attended (6515, 4)


** Data Glimpse **

In [3]:
print train.head(2)
print test.head(2)

   Patient_ID  Health_Camp_ID Registration_Date  Var1  Var2  Var3  Var4  Var5
0      489652            6578         10-Sep-05     4     0     0     0     2
1      507246            6578         18-Aug-05    45     5     0     0     7
   Patient_ID  Health_Camp_ID Registration_Date  Var1  Var2  Var3  Var4  Var5
0      505701            6548         21-May-06     1     0     0     0     2
1      500633            6584         02-Jun-06     0     0     0     0     0


In [4]:
print health_camp_detail.head(2)
print "\n"
print patient_profile.head(2)

   Health_Camp_ID Camp_Start_Date Camp_End_Date Category1 Category2  Category3
0            6560       16-Aug-03     20-Aug-03     First         B          2
1            6530       16-Aug-03     28-Oct-03     First         C          2


   Patient_ID  Online_Follower  LinkedIn_Shared  Twitter_Shared  \
0      516956                0                0               0   
1      507733                0                0               0   

   Facebook_Shared Income Education_Score Age First_Interaction City_Type  \
0                0      1              90  39         18-Jun-03       NaN   
1                0      1            None  40         20-Jul-03         H   

   Employer_Category  
0  Software Industry  
1  Software Industry  


In [5]:
first_health_camp_attended.head(2)
first_health_camp_attended = first_health_camp_attended[[0,1,2,3]]
first_health_camp_attended.head(2)

,Patient_ID,Health_Camp_ID,Donation,Health_Score
0,506181,6560,40,0.439024
1,494977,6560,20,0.097561


In [6]:
print second_health_camp_attended.head(2)
print "\n"
print third_health_camp_attended.head(2)

   Patient_ID  Health_Camp_ID  Health Score
0      526631            6536      0.875136
1      509122            6536      0.755700


   Patient_ID  Health_Camp_ID  Number_of_stall_visited  \
0      517875            6527                        3   
1      504692            6578                        1   

   Last_Stall_Visited_Number  
0                          1  
1                          1  


** Creating combined_df (train+test) **

In [7]:
def create_combined_df(train, test):
    '''
    Creates combined dataframe from train & test dataframes.
    Basically concats train & test dataframes.
    
    Parameters
    -----------
    train : a pandas DataFrame.
    test : a pandas DataFrame.
    
    Returns
    --------
    combined_df : concated pandas dataframe.
    '''
    train['train_or_test'] = 'train'
    test['train_or_test'] = 'test'
    combined_df = pd.concat([train,test])
    return combined_df

In [8]:
# Combining Train & Test
combined_df = create_combined_df(train, test)
print combined_df.shape
combined_df.head(2)

(110527, 9)


,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,train_or_test
0,489652,6578,10-Sep-05,4,0,0,0,2,train
1,507246,6578,18-Aug-05,45,5,0,0,7,train


** Generating the outcome **

In [9]:
col_names = [['Patient_ID','Health_Camp_ID','Outcome']]
first_health_camp_attended = first_health_camp_attended[['Patient_ID','Health_Camp_ID','Health_Score']]
first_health_camp_attended.columns = col_names
second_health_camp_attended = second_health_camp_attended[['Patient_ID','Health_Camp_ID','Health Score']]
second_health_camp_attended.columns = col_names
third_health_camp_attended = third_health_camp_attended[['Patient_ID','Health_Camp_ID','Number_of_stall_visited']]
third_health_camp_attended = third_health_camp_attended[third_health_camp_attended['Number_of_stall_visited']>0]
third_health_camp_attended.columns = col_names
all_camps = pd.concat([first_health_camp_attended, second_health_camp_attended, third_health_camp_attended])
all_camps['Outcome'] = 1
print all_camps.shape

(20534, 3)


In [10]:
combined_df = combined_df.merge(all_camps, on=['Patient_ID','Health_Camp_ID'], how='left')
combined_df['Outcome'] = combined_df['Outcome'].fillna(0).astype('int')
combined_df.head(2)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,train_or_test,Outcome
0,489652,6578,10-Sep-05,4,0,0,0,2,train,1
1,507246,6578,18-Aug-05,45,5,0,0,7,train,0


In [11]:
print combined_df[combined_df['train_or_test']=='train'].Outcome.value_counts()
print combined_df.train_or_test.value_counts()

0    54744
1    20534
Name: Outcome, dtype: int64
train    75278
test     35249
Name: train_or_test, dtype: int64


i.e. ~27% of people who register for the camp, turns up!!

In [12]:
# Just checking
# Registration_Date is missing only in train
print combined_df[combined_df.Registration_Date.isnull()]['train_or_test'].unique()
print combined_df[combined_df.isnull().any(axis=1)]['train_or_test'].unique()

['train']
['train']


** Merging with health_camp_detail data **

In [13]:
combined_df = combined_df.merge(health_camp_detail, on=['Health_Camp_ID'], how='left')
print combined_df.shape
combined_df.head(2)

(110527, 15)


,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,train_or_test,Outcome,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,489652,6578,10-Sep-05,4,0,0,0,2,train,1,16-Aug-05,14-Oct-05,Third,G,2
1,507246,6578,18-Aug-05,45,5,0,0,7,train,0,16-Aug-05,14-Oct-05,Third,G,2


** Merging with patient_profile data **

In [14]:
combined_df = combined_df.merge(patient_profile, on=['Patient_ID'], how='left')
print combined_df.shape
combined_df.head(2)

(110527, 25)


,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,train_or_test,Outcome,...,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,489652,6578,10-Sep-05,4,0,0,0,2,train,1,...,0,0,0,0,None,None,None,06-Dec-04,NaN,NaN
1,507246,6578,18-Aug-05,45,5,0,0,7,train,0,...,0,0,0,0,1,75,40,08-Sep-04,C,Others


In [15]:
combined_df[combined_df.train_or_test=='train'].Patient_ID.value_counts().head()

516956    32
490196    28
513633    28
509188    28
517006    25
Name: Patient_ID, dtype: int64

In [16]:
patients_train = combined_df[combined_df.train_or_test=='train']['Patient_ID'].unique()
patients_test = combined_df[combined_df.train_or_test=='test']['Patient_ID'].unique()
patients_diff = np.setdiff1d(patients_test, patients_train, assume_unique=True)
print "No of Unique Patients that are in test but not in train:", len(patients_diff)
print patients_diff
print "No of Unique Patients that are in train:", len(patients_train)
print "No of Unique Patients that are in test:", len(patients_test)

No of Unique Patients that are in test but not in train: 6864
[497447 496446 494067 ..., 508554 527157 496733]
No of Unique Patients that are in train: 29828
No of Unique Patients that are in test: 15324


So nearly 55% of patients are in test that are also in train.            
So mean_outcome_patientwise can help a bit!! 

In [17]:
mean_outcome_patientwise = combined_df[combined_df.train_or_test=='train'].groupby(['Patient_ID'])[['Outcome']].mean()
mean_outcome_patientwise.reset_index(inplace=True)
mean_outcome_patientwise.rename(columns={'Outcome':'mean_outcome_pwise'}, inplace=True)
print mean_outcome_patientwise.shape
mean_outcome_patientwise.head()

(29828, 2)


,Patient_ID,mean_outcome_pwise
0,485679,0.5
1,485680,0.0
2,485681,0.0
3,485682,1.0
4,485684,0.0


In [18]:
# Merging mean_outcome_patientwise with combined_df
print combined_df.shape
combined_df = combined_df.merge(mean_outcome_patientwise, on=['Patient_ID'], how='left')
print combined_df.shape
combined_df.head(2)

(110527, 25)
(110527, 26)


,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,train_or_test,Outcome,...,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,mean_outcome_pwise
0,489652,6578,10-Sep-05,4,0,0,0,2,train,1,...,0,0,0,None,None,None,06-Dec-04,NaN,NaN,0.2
1,507246,6578,18-Aug-05,45,5,0,0,7,train,0,...,0,0,0,1,75,40,08-Sep-04,C,Others,0.5


In [102]:
# Grouping patient on the basis of City_Type
#combined_df.replace('None','',inplace=True)
#combined_df[combined_df['Age'].notnull()].groupby(['City_Type'])[['Age']].median()

In [19]:
def extract_from_date(date):
    date_fields = []
    if type(date) is np.float:
        date_fields = [-999,-999,-999] 
    else:
        datetime_object = datetime.strptime(date, '%d-%b-%y')
        date_fields = [datetime_object.day, datetime_object.month, datetime_object.year]
    return date_fields

In [20]:
combined_df['Reg_Date'] = combined_df['Registration_Date'].apply(extract_from_date)
combined_df['Reg_Day'] = combined_df['Reg_Date'].apply(lambda x: x[0])
combined_df['Reg_Month'] = combined_df['Reg_Date'].apply(lambda x: x[1])
combined_df['Reg_Year'] = combined_df['Reg_Date'].apply(lambda x: x[2])

combined_df['Camp_Start_Date'] = combined_df['Camp_Start_Date'].apply(extract_from_date)
combined_df['Camp_Start_Day'] = combined_df['Camp_Start_Date'].apply(lambda x: x[0])
combined_df['Camp_Start_Month'] = combined_df['Camp_Start_Date'].apply(lambda x: x[1])
combined_df['Camp_Start_Year'] = combined_df['Camp_Start_Date'].apply(lambda x: x[2])

combined_df['Camp_End_Date'] = combined_df['Camp_End_Date'].apply(extract_from_date)
combined_df['Camp_End_Day'] = combined_df['Camp_End_Date'].apply(lambda x: x[0])
combined_df['Camp_End_Month'] = combined_df['Camp_End_Date'].apply(lambda x: x[1])
combined_df['Camp_End_Year'] = combined_df['Camp_End_Date'].apply(lambda x: x[2])

combined_df['First_Interaction'] = combined_df['First_Interaction'].apply(extract_from_date)
combined_df['First_Interaction_Day'] = combined_df['First_Interaction'].apply(lambda x: x[0])
combined_df['First_Interaction_Month'] = combined_df['First_Interaction'].apply(lambda x: x[1])
combined_df['First_Interaction_Year'] = combined_df['First_Interaction'].apply(lambda x: x[2])

combined_df.drop(['Registration_Date','Reg_Date','Camp_Start_Date','Camp_End_Date','First_Interaction'], axis=1, inplace=True)
print combined_df.shape
combined_df.head(2)

(110527, 34)


,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,train_or_test,Outcome,Category1,...,Reg_Year,Camp_Start_Day,Camp_Start_Month,Camp_Start_Year,Camp_End_Day,Camp_End_Month,Camp_End_Year,First_Interaction_Day,First_Interaction_Month,First_Interaction_Year
0,489652,6578,4,0,0,0,2,train,1,Third,...,2005,16,8,2005,14,10,2005,6,12,2004
1,507246,6578,45,5,0,0,7,train,0,Third,...,2005,16,8,2005,14,10,2005,8,9,2004


In [21]:
# Checking whether train data is from camps that started before April 2006 or not
combined_df[combined_df.train_or_test=='train']['Camp_Start_Year'].unique()
# Year checked!

array([2005, 2004, 2006, 2003], dtype=int64)

** Merging first_health_camp_attended, second_health_camp_attended, third_health_camp_attended **

In [22]:
'''
combined_df = combined_df.merge(first_health_camp_attended[[0,1,2,3]], on=['Patient_ID','Health_Camp_ID'], how='left')
combined_df.rename(columns={'Health_Score':'Health_Score_Camp1'}, inplace=True)
combined_df = combined_df.merge(second_health_camp_attended, on=['Patient_ID','Health_Camp_ID'], how='left')
combined_df.rename(columns={'Health_Score':'Health_Score_Camp2'}, inplace=True)
combined_df = combined_df.merge(third_health_camp_attended, on=['Patient_ID','Health_Camp_ID'], how='left')
print combined_df.shape
combined_df.head(2)
'''

"\ncombined_df = combined_df.merge(first_health_camp_attended[[0,1,2,3]], on=['Patient_ID','Health_Camp_ID'], how='left')\ncombined_df.rename(columns={'Health_Score':'Health_Score_Camp1'}, inplace=True)\ncombined_df = combined_df.merge(second_health_camp_attended, on=['Patient_ID','Health_Camp_ID'], how='left')\ncombined_df.rename(columns={'Health_Score':'Health_Score_Camp2'}, inplace=True)\ncombined_df = combined_df.merge(third_health_camp_attended, on=['Patient_ID','Health_Camp_ID'], how='left')\nprint combined_df.shape\ncombined_df.head(2)\n"

In [23]:
print health_camp_detail.Category1.value_counts()
print health_camp_detail.Category2.value_counts()
print health_camp_detail.Category3.value_counts()

First     44
Second    11
Third     10
Name: Category1, dtype: int64
C    16
E    15
F    11
G    10
A     7
D     4
B     2
Name: Category2, dtype: int64
2    62
1     3
Name: Category3, dtype: int64


In [24]:
# OHE of Category1, Category2, Category3
print combined_df.shape
combined_df = pd.get_dummies(combined_df, columns=['Category1','Category2','Category3'])
print combined_df.shape
print combined_df.columns.tolist()

(110527, 34)
(110527, 43)
['Patient_ID', 'Health_Camp_ID', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'train_or_test', 'Outcome', 'Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared', 'Facebook_Shared', 'Income', 'Education_Score', 'Age', 'City_Type', 'Employer_Category', 'mean_outcome_pwise', 'Reg_Day', 'Reg_Month', 'Reg_Year', 'Camp_Start_Day', 'Camp_Start_Month', 'Camp_Start_Year', 'Camp_End_Day', 'Camp_End_Month', 'Camp_End_Year', 'First_Interaction_Day', 'First_Interaction_Month', 'First_Interaction_Year', 'Category1_First', 'Category1_Second', 'Category1_Third', 'Category2_A', 'Category2_B', 'Category2_C', 'Category2_D', 'Category2_E', 'Category2_F', 'Category2_G', 'Category3_1', 'Category3_2']


In [25]:
print combined_df.Employer_Category.value_counts()
print combined_df.City_Type.value_counts()

Technology           4352
Software Industry    2938
Others               2937
Consulting           2755
Education            1487
BFSI                 1118
Retail                707
Manufacturing         666
Health                593
Transport             492
Real Estate           462
Food                  431
Telecom               370
Broadcasting          256
Name: Employer_Category, dtype: int64
B    12242
H    10203
D     7881
C     6343
G     6298
E     6009
A     5905
I     4904
F     4140
Name: City_Type, dtype: int64


In [26]:
# OHE of City_Type, Employer_Category
print combined_df.shape
combined_df = pd.get_dummies(combined_df, columns=['City_Type','Employer_Category'], dummy_na=True)
print combined_df.shape
print combined_df.columns.tolist()

(110527, 43)
(110527, 66)
['Patient_ID', 'Health_Camp_ID', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'train_or_test', 'Outcome', 'Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared', 'Facebook_Shared', 'Income', 'Education_Score', 'Age', 'mean_outcome_pwise', 'Reg_Day', 'Reg_Month', 'Reg_Year', 'Camp_Start_Day', 'Camp_Start_Month', 'Camp_Start_Year', 'Camp_End_Day', 'Camp_End_Month', 'Camp_End_Year', 'First_Interaction_Day', 'First_Interaction_Month', 'First_Interaction_Year', 'Category1_First', 'Category1_Second', 'Category1_Third', 'Category2_A', 'Category2_B', 'Category2_C', 'Category2_D', 'Category2_E', 'Category2_F', 'Category2_G', 'Category3_1', 'Category3_2', 'City_Type_A', 'City_Type_B', 'City_Type_C', 'City_Type_D', 'City_Type_E', 'City_Type_F', 'City_Type_G', 'City_Type_H', 'City_Type_I', 'City_Type_nan', 'Employer_Category_BFSI', 'Employer_Category_Broadcasting', 'Employer_Category_Consulting', 'Employer_Category_Education', 'Employer_Category_Food', 'Employer_Category_Healt

In [27]:
# OHE of 'Reg_Day', 'Reg_Month', 'Reg_Year', 'Camp_Start_Day', 'Camp_Start_Month', 'Camp_Start_Year', 
# 'Camp_End_Day', 'Camp_End_Month', 'Camp_End_Year', 'First_Interaction_Day', 'First_Interaction_Month', 'First_Interaction_Year'
col_list = ['Reg_Day','Reg_Month','Reg_Year','Camp_Start_Day','Camp_Start_Month','Camp_Start_Year','Camp_End_Day','Camp_End_Month','Camp_End_Year','First_Interaction_Day','First_Interaction_Month','First_Interaction_Year']
no_of_new_cols = 0
for col in col_list:
    no_of_new_cols += len(combined_df[col].unique())
print no_of_new_cols
print combined_df.shape
combined_df = pd.get_dummies(combined_df, columns=col_list, dummy_na=True)
print combined_df.shape
print combined_df.columns.tolist()

189
(110527, 66)
(110527, 255)
['Patient_ID', 'Health_Camp_ID', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'train_or_test', 'Outcome', 'Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared', 'Facebook_Shared', 'Income', 'Education_Score', 'Age', 'mean_outcome_pwise', 'Category1_First', 'Category1_Second', 'Category1_Third', 'Category2_A', 'Category2_B', 'Category2_C', 'Category2_D', 'Category2_E', 'Category2_F', 'Category2_G', 'Category3_1', 'Category3_2', 'City_Type_A', 'City_Type_B', 'City_Type_C', 'City_Type_D', 'City_Type_E', 'City_Type_F', 'City_Type_G', 'City_Type_H', 'City_Type_I', 'City_Type_nan', 'Employer_Category_BFSI', 'Employer_Category_Broadcasting', 'Employer_Category_Consulting', 'Employer_Category_Education', 'Employer_Category_Food', 'Employer_Category_Health', 'Employer_Category_Manufacturing', 'Employer_Category_Others', 'Employer_Category_Real Estate', 'Employer_Category_Retail', 'Employer_Category_Software Industry', 'Employer_Category_Technology', 'Employer_Category

In [28]:
combined_df.fillna(-999,inplace=True)
combined_df.replace('None',-999,inplace=True)
combined_df.head(2)

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,train_or_test,Outcome,Online_Follower,...,First_Interaction_Month_11.0,First_Interaction_Month_12.0,First_Interaction_Month_nan,First_Interaction_Year_2002.0,First_Interaction_Year_2003.0,First_Interaction_Year_2004.0,First_Interaction_Year_2005.0,First_Interaction_Year_2006.0,First_Interaction_Year_2007.0,First_Interaction_Year_nan
0,489652,6578,4,0,0,0,2,train,1,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,507246,6578,45,5,0,0,7,train,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


# XGB Model

In [29]:
train = combined_df[combined_df['train_or_test']=='train']
test = combined_df[combined_df['train_or_test']=='test']
print train.shape
print test.shape

(75278, 255)
(35249, 255)


In [30]:
validation = train[train['Camp_Start_Year_2006.0']==1.0]
train2 = train[train['Camp_Start_Year_2006.0']==0.0]

# These corresponds to the Health Camp that started between 01st Sept 2005 - 31st March 2006
#Health_Camp_ID_list_for_val = [6555,6543,6537,6571,6575,6534,6541,6528,6529]
#validation = train[train.Health_Camp_ID in Health_Camp_ID_list_for_val]
#train2 = train[train.Health_Camp_ID not in Health_Camp_ID_list_for_val]

print validation.shape
print train2.shape
print train2.shape[0]+validation.shape[0]

(5567, 255)
(69711, 255)
75278


In [31]:
y_train = train.Outcome.values
y_train2 = train2.Outcome.values
y_validation = validation.Outcome.values
train.drop(['train_or_test','Patient_ID','Health_Camp_ID','Outcome'], axis=1, inplace=True)
train2.drop(['train_or_test','Patient_ID','Health_Camp_ID','Outcome'], axis=1, inplace=True)
validation.drop(['train_or_test','Patient_ID','Health_Camp_ID','Outcome'], axis=1, inplace=True)
sub_df = test[['Patient_ID','Health_Camp_ID']].reset_index(drop=True)
test.drop(['train_or_test','Patient_ID','Health_Camp_ID','Outcome'], axis=1, inplace=True)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

In [32]:
print len(y_train), y_train
print len(y_validation), y_validation
print sub_df.head(2)
print train.head(2)
print validation.head(2)
print test.head(2)

75278 [1 0 1 ..., 1 0 0]
5567 [1 1 0 ..., 1 0 1]
   Patient_ID  Health_Camp_ID
0      505701            6548
1      500633            6584
   Var1  Var2  Var3  Var4  Var5  Online_Follower  LinkedIn_Shared  \
0     4     0     0     0     2                0                0   
1    45     5     0     0     7                0                0   

   Twitter_Shared  Facebook_Shared Income             ...              \
0               0                0   -999             ...               
1               0                0      1             ...               

  First_Interaction_Month_11.0 First_Interaction_Month_12.0  \
0                          0.0                          1.0   
1                          0.0                          0.0   

   First_Interaction_Month_nan  First_Interaction_Year_2002.0  \
0                          0.0                            0.0   
1                          0.0                            0.0   

   First_Interaction_Year_2003.0  First_Interac

In [33]:
dtrain = xgb.DMatrix(train.values, label=y_train)
print dtrain.num_row()
print dtrain.num_col()

75278
251


In [34]:
dtrain2 = xgb.DMatrix(train2.values, label=y_train2)
print dtrain2.num_row()
print dtrain2.num_col()

69711
251


In [35]:
dval = xgb.DMatrix(validation.values, label=y_validation)
print dval.num_row()
print dval.num_col()

5567
251


In [36]:
dtest = xgb.DMatrix(test.values)
print dtest.num_row()
print dtest.num_col()

35249
251


In [37]:
# XGB Params
param = {'max_depth':6, 
         'eta':0.01,
         'subsample':0.9,
         'colsample_bytree':0.3,
         #'min_child_weight':0,
         'gamma':0.05,
         'eval_metric':'auc',
         'objective':'binary:logistic',
         'booster':'gbtree',
         'seed':1729,
         #'nthread':7,
         'silent':1}

In [38]:
watchlist2 = [(dval,'validation')]
num_train_round = 2000
early_stopping_rounds = 200

** Validation **

In [ ]:
def time_elapsed(t0):
    return (time.time()-t0)/60

In [39]:
t0 = time.time()
print num_train_round
xgbModel2 = xgb.train(param, 
                     dtrain2, 
                     num_boost_round=num_train_round,
                     early_stopping_rounds=early_stopping_rounds,
                     evals=watchlist2, 
                     maximize=True, 
                     verbose_eval=50)
print time_elapsed(t0), "minutes elapsed!"

2000
[0]	validation-auc:0.502775
Will train until validation-auc hasn't improved in 200 rounds.
[50]	validation-auc:0.901708
[100]	validation-auc:0.90642
[150]	validation-auc:0.90654
[200]	validation-auc:0.906057
[250]	validation-auc:0.905445
[300]	validation-auc:0.903955
Stopping. Best iteration:
[148]	validation-auc:0.906782



NameError: name 'time_elapsed' is not defined

** Feature Importance **

In [40]:
def importance_XGB(xgbModel, cols_list):
    impdf = []
    for ft, score in xgbModel.get_fscore().iteritems():
        impdf.append({'feature': ft, 'feature_name': cols_list[int(ft[1:])], 'importance': score})
    impdf = pd.DataFrame(impdf)
    impdf = impdf.sort_values(by='importance', ascending=False).reset_index(drop=True)
    impdf['importance'] /= impdf['importance'].sum()
    return impdf

In [42]:
impdf = importance_XGB(xgbModel2, train.columns)
impdf

,feature,feature_name,importance
0,f12,mean_outcome_pwise,0.082083
1,f11,Age,0.033402
2,f10,Education_Score,0.030781
3,f9,Income,0.027380
4,f0,Var1,0.025149
5,f4,Var5,0.016729
6,f34,City_Type_nan,0.015502
7,f247,First_Interaction_Year_2005.0,0.015335
8,f245,First_Interaction_Year_2003.0,0.013104
9,f13,Category1_First,0.011153


** Training on full train data **

In [43]:
t0 = time.time()
num_train_round = 50
watchlist = [(dtrain,'train')]
xgbModel = xgb.train(param, 
                     dtrain, 
                     num_boost_round=num_train_round,
                     evals=watchlist, 
                     maximize=True, 
                     verbose_eval=15)
print time_elapsed(t0), "minutes elapsed!"

[0]	train-auc:0.945226
[15]	train-auc:0.91683
[30]	train-auc:0.929288
[45]	train-auc:0.930544


NameError: name 'time_elapsed' is not defined

** Prediction **

In [44]:
pred = xgbModel.predict(dtest)
pred

array([ 0.55640322,  0.50901914,  0.36122817, ...,  0.41586927,
        0.39609927,  0.54184848], dtype=float32)

In [45]:
sub_df['Outcome'] = pred
print sub_df.shape
sub_df.head(3)

(35249, 3)


,Patient_ID,Health_Camp_ID,Outcome
0,505701,6548,0.556403
1,500633,6584,0.509019
2,506945,6582,0.361228


In [46]:
# Write csv pred file
sub_df.to_csv('D:\\Data Science\\AV\\Knocktober\\submissions\\sub_meanoutcome_pwise.csv', index=False)